In [1]:
# django shell 기능을 jupyter notebook을 이용해 작업
# - django 프로젝트 ROOT 디렉토리에서 주피터노트북을 실행
# - jupyter notebook에서 django shell을 사용하기 위해서 환경설정
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'  # root/config/settings.py 경로 지정.
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [2]:
from poll.models import Question, Choice

In [3]:
mm = Question.objects
print(type(mm))

<class 'django.db.models.manager.Manager'>


In [4]:
mm.all()

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>, <Question: 싫어하는 동물은 무엇입니까?>]>

모델클래스.objects.SQL메소드()[.SQL메소드()...]

# 조회(select)
- all(): 전체조회

In [5]:
qs = Question.objects.all()  # select * from question
# 조회결과는 QuerySet 객체 반환
# QuestSet - 조회결과를 조회, 추가 SQL문을 실행
print(type(qs))
print(qs.query)

<class 'django.db.models.query.QuerySet'>
SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question"


In [6]:
qs  # iterable

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>, <Question: 싫어하는 동물은 무엇입니까?>]>

In [7]:
for m in qs:
    print(type(m), m, "--", m.question_text, m.pub_date, m.id, m.pk)  # model.pk : primary 컬럼의 값을 조회

<class 'poll.models.Question'> 좋아하는 색깔은 무엇입니까? -- 좋아하는 색깔은 무엇입니까? 2021-10-05 08:17:35.161348+00:00 1 1
<class 'poll.models.Question'> 좋아하는 동물은 무엇입니까? -- 좋아하는 동물은 무엇입니까? 2021-10-05 08:17:48.995562+00:00 2 2
<class 'poll.models.Question'> 가고 싶은 나라를 선택하세요. -- 가고 싶은 나라를 선택하세요. 2021-10-05 08:17:59.136089+00:00 3 3
<class 'poll.models.Question'> 싫어하는 동물은 무엇입니까? -- 싫어하는 동물은 무엇입니까? 2021-10-05 08:18:11.234816+00:00 4 4


In [8]:
# QuerySet: iterable - 반복문으로 조회, indexing, slicing

In [9]:
qs[2]
print("id:", qs[2].pk, "등록일시", qs[2].pub_date, ",질문:", qs[3].question_text)

id: 3 등록일시 2021-10-05 08:17:59.136089+00:00 ,질문: 싫어하는 동물은 무엇입니까?


In [10]:
# 음수 indexing은 지원 안한다.
# qs[-1]

In [11]:
qs[1:4]

[<Question: 좋아하는 동물은 무엇입니까?>,
 <Question: 가고 싶은 나라를 선택하세요.>,
 <Question: 싫어하는 동물은 무엇입니까?>]

In [12]:
qs[::2]

[<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>]

In [13]:
qs.first()  # 조회결과 중 첫번째 것 반환

<Question: 좋아하는 색깔은 무엇입니까?>

In [14]:
qs.last()  # 조회결과 중 마지막 것 반환

<Question: 싫어하는 동물은 무엇입니까?>

## where 절 이용한 조회
조회조건이 있는 조회
- filter(조회조건): 조건을 만족하는 행이 0개 이상일 경우
- exclude(조회조건): filter()반대. 조건을 만족하지 않는 0개이상의 행동을 조회
- get(조회조건): 조회조건을 만족하는 1개 행 조회. PK 로 조회할 때 사용. 조회결과가 0개이거나 2개 이상인 경우 Exception 발생.
- filter/exclude() 반환타입: QuerySet
- get() 반환타입: Model객체 반환

In [15]:
# pk(id) = 2
qs = Question.objects.filter(id=2)
r = qs.first()
r.id, r.question_text, r.pub_date

(2,
 '좋아하는 동물은 무엇입니까?',
 datetime.datetime(2021, 10, 5, 8, 17, 48, 995562, tzinfo=<UTC>))

In [16]:
question = Question.objects.get(pk=2)
print(type(question))
question.pk, question.question_text

<class 'poll.models.Question'>


(2, '좋아하는 동물은 무엇입니까?')

In [17]:
qs = Question.objects.exclude(pk=2)  # where pk <> 2
print(qs.query)
qs

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE NOT ("poll_question"."id" = 2)


<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>, <Question: 싫어하는 동물은 무엇입니까?>]>

In [18]:
try:
    q = Question.objects.get(pk=100)  # 없는 것 조회: DoesNotExist Exception 발생
#     추가작업
except:
    print("id가 100인 질문은 없습니다.")

id가 100인 질문은 없습니다.


In [19]:
Question.objects.filter(pk=100)

<QuerySet []>

In [20]:
q = Question.objects.get(pk__gt=1)  # where pk > 1 : 1개 이상이 조회되는 경우 get()-MultipleObjectsReturned Exception 발생

MultipleObjectsReturned: get() returned more than one Question -- it returned 3!

In [21]:
# 조회조건
qs = Question.objects.filter(pk__lt=3)  # pk < 3
qs = Question.objects.filter(pk__lte=3)  # pk <= 3
qs = Question.objects.filter(pk__gt=3)  # pk > 3
qs = Question.objects.filter(pk__gte=3)  # pk >= 3
qs = Question.objects.filter(pk__in=[1,4,6])  # pk in (1,4,6)
qs = Question.objects.filter(pk__range=[2,5])  # pk between 2 and 5-1
qs = Question.objects.filter(question_text__startswith="좋아하는")  # question_text like '좋아하는%' 좋아하는 으로 시작하는
qs = Question.objects.filter(question_text__endswith='무엇입니까?')  # question_text like '%무엇입니까?' 무엇입니까로 끝나는
qs = Question.objects.filter(question_text__contains='동물')  # question_text like '%동물%' 동물이 들어간
print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE "poll_question"."question_text" LIKE %동물% ESCAPE '\'
2 좋아하는 동물은 무엇입니까?
4 싫어하는 동물은 무엇입니까?


### AND로 조건을 묶는 경우
- 조회함수들의 매개변수에 조건들을 나열해 준다.

In [22]:
# pk = 2 and q_text가 "좋아하는으로 시작"
qs = Question.objects.filter(pk=2, question_text__startswith='좋아하는')

print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE ("poll_question"."id" = 2 AND "poll_question"."question_text" LIKE 좋아하는% ESCAPE '\')
2 좋아하는 동물은 무엇입니까?


### OR로 조건들을 묶는 경우
- 개별 조건을 Q()에 넣고 `|` 사용

In [23]:
# pk > 3 or question_text like '%어디인가요?'
from django.db.models import Q

qs = Question.objects.filter(Q(pk__lt=3) | Q(question_text__endswith='어디인가요?'))

print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE ("poll_question"."id" < 3 OR "poll_question"."question_text" LIKE %어디인가요? ESCAPE '\')
1 좋아하는 색깔은 무엇입니까?
2 좋아하는 동물은 무엇입니까?


### 컬럼선택, 정렬
- values(조회할컬럼변수나열): 컬럼 선택
    - 조회한 데이터를 directory에 넣어서 반환 - QuerySet[{},{}]
- order_by(기준컬럼): 정렬-ASC(기본) "-컬럼명": DESC

In [24]:
Choice.objects.all()

<QuerySet [<Choice: 검정색>, <Choice: 빨간색>, <Choice: 뱀>, <Choice: 쥐>, <Choice: 고라니>, <Choice: 악어>, <Choice: 보라색>, <Choice: 민트색>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 원숭이>, <Choice: 판다>, <Choice: 독일>, <Choice: 영국>, <Choice: 스페인>, <Choice: 이탈리아>]>

In [27]:
# qs = Choice.objects.all().values('choice_text', 'vote')  # select choice_text, vote  from choice
qs = Choice.objects.values('Choice_text', 'vote').all()
print(qs.query)
print(qs)
for m in qs:
# #     print(m.pk, m.choice_text, m.vote, m.question, type(m.question), sep='\t')
    print(m['Choice_text'], m['vote'], sep='\t')

SELECT "poll_choice"."Choice_text", "poll_choice"."vote" FROM "poll_choice"
<QuerySet [{'Choice_text': '검정색', 'vote': 0}, {'Choice_text': '빨간색', 'vote': 0}, {'Choice_text': '뱀', 'vote': 0}, {'Choice_text': '쥐', 'vote': 3}, {'Choice_text': '고라니', 'vote': 8}, {'Choice_text': '악어', 'vote': 6}, {'Choice_text': '보라색', 'vote': 2}, {'Choice_text': '민트색', 'vote': 14}, {'Choice_text': '강아지', 'vote': 13}, {'Choice_text': '고양이', 'vote': 6}, {'Choice_text': '원숭이', 'vote': 3}, {'Choice_text': '판다', 'vote': 6}, {'Choice_text': '독일', 'vote': 3}, {'Choice_text': '영국', 'vote': 3}, {'Choice_text': '스페인', 'vote': 22}, {'Choice_text': '이탈리아', 'vote': 4}]>
검정색	0
빨간색	0
뱀	0
쥐	3
고라니	8
악어	6
보라색	2
민트색	14
강아지	13
고양이	6
원숭이	3
판다	6
독일	3
영국	3
스페인	22
이탈리아	4


In [31]:
# qs = Choice.objects.values('id', 'Choice_text').all().order_by('choice_text')  # choice_text를 기준으로 ASC 정렬
# qs = Choice.objects.values('id', 'Choice_text').all().order_by('-choice_text')  # choice_text를 기준으로 DESC 정렬
# qs = Choice.objects.values('id', 'Choice_text').all().order_by('-choice_text', 'id')
qs = Choice.objects.values('id', 'Choice_text').filter(pk__lt=7).order_by('-Choice_text', 'id')
print(qs.query)
for m in qs:
    print(m['id'], m['Choice_text'])

SELECT "poll_choice"."id", "poll_choice"."Choice_text" FROM "poll_choice" WHERE "poll_choice"."id" < 7 ORDER BY "poll_choice"."Choice_text" DESC, "poll_choice"."id" ASC
5 쥐
2 빨간색
4 뱀
6 고라니
1 검정색


select 컬럼명 -values() from 테이블명 - 모델클래스명 where 조건 - filter, exclude, get 메소드<br>
order by 정렬조건 - order_by()메소드

### 집계함수

In [32]:
# 조회한 데이터에 개수 조회
# QuerySet.count(), len(QuerySet)
qs = Choice.objects.all()
print(qs.count(), len(qs))

16 16


- 전체 데이터셋 기준 집계
    - aggregate(집계함수(), 집계함수()...)
- Groupby 집계
    - values("나눌기준컬럼명", ..).annotate(집계함수(), ..)
- 결과를 dictionary에 넣어서 반환

In [33]:
# 집계함수들 import
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

In [36]:
# 전체 데이터셋 기준 집계
# select count(vote)
result = Choice.objects.aggregate(Count('vote'), Sum('vote'), Min('vote'), Max('vote'), StdDev('vote'), Variance('vote'))
result

{'vote__count': 16,
 'vote__sum': 93,
 'vote__min': 0,
 'vote__max': 22,
 'vote__stddev': 5.790280109804706,
 'vote__variance': 33.52734375}

In [51]:
# 결과 key: field이름__집계이름
# result.keys()
result['vote__avg']

KeyError: 'vote__avg'

In [39]:
# select min(pub_date)
# datetime. data. time 타입: min-과거, max-미래
result2 = Question.objects.aggregate(Min('pub_date'), Max('pub_date'))
result2

{'pub_date__min': datetime.datetime(2021, 10, 5, 8, 17, 35, 161348, tzinfo=<UTC>),
 'pub_date__max': datetime.datetime(2021, 10, 5, 8, 18, 11, 234816, tzinfo=<UTC>)}

In [42]:
min_date = result2['pub_date__min']
print(type(min_date))
min_date.year, min_date.month, min_date.day, min_date.hour, min_date.minute, min_date.second

<class 'datetime.datetime'>


(2021, 10, 5, 8, 17, 35)

In [47]:
# datetime.strftime("형식문자") %Y:연도, %m:월, %d:일, %H: 시간(24), %M: 분, %S: 초 => 원하는 형식의 문자열로 변환
r = min_date.strftime("%Y년 %m월 %d일 %H:%M:%S")
print(r, type(r))

2021년 10월 05일 08:17:35 <class 'str'>


In [52]:
# groupby
# select sum(vote), ... from choice groupby question
result3 = Choice.objects.values('question').annotate(Sum("vote"), Avg('vote'), Max('Choice_text'))  # 문자열 min: a, max: z
for d in result3:
    print(d)

{'question': 1, 'vote__sum': 16, 'vote__avg': 4.0, 'Choice_text__max': '빨간색'}
{'question': 2, 'vote__sum': 28, 'vote__avg': 7.0, 'Choice_text__max': '판다'}
{'question': 3, 'vote__sum': 32, 'vote__avg': 8.0, 'Choice_text__max': '이탈리아'}
{'question': 4, 'vote__sum': 17, 'vote__avg': 4.25, 'Choice_text__max': '쥐'}


In [53]:
# having절 처리 함수는 없다.
# select sum(vote),... from choice groupby question having sum(vote)>20
for d in result3:
    if d['vote__sum'] > 20:  # having절 조건
        print(d)

{'question': 2, 'vote__sum': 28, 'vote__avg': 7.0, 'Choice_text__max': '판다'}
{'question': 3, 'vote__sum': 32, 'vote__avg': 8.0, 'Choice_text__max': '이탈리아'}


## 테이블간의 관계

### JOIN 을 이용한 조회

#### 자식테이블에서 부모테이블값을 조회

- Question: 부모
- Choice: 자식

In [57]:
c = Choice.objects.get(pk=1)
type(c), c
c.Choice_text, c.vote

('검정색', 0)

In [56]:
# pk가 1인 보기의 질문?
q = c.question
type(q), q.pk, q.question_text, q.pub_date

(poll.models.Question,
 1,
 '좋아하는 색깔은 무엇입니까?',
 datetime.datetime(2021, 10, 5, 8, 17, 35, 161348, tzinfo=<UTC>))

In [63]:
# pk가 2인 choice(보기)의 보기내용, vote, 질문
try:
    id = 2
    result_choice = Choice.objects.get(pk=id)
    print(result_choice.Choice_text, result_choice.vote, result_choice.question.question_text)
#     result_choice.question: Question 객체
except:
    print(f"{id}는 없는 보기입니다.")

빨간색 0 좋아하는 색깔은 무엇입니까?


In [64]:
# pk 2 ~ 5 인 보기의 text, vote 질문
result = Choice.objects.filter(pk__range=[2,5])
for choice in result:
    print(choice.pk, choice.Choice_text, choice.vote, choice.question.question_text, sep='\t')

2	빨간색	0	좋아하는 색깔은 무엇입니까?
4	뱀	0	싫어하는 동물은 무엇입니까?
5	쥐	3	싫어하는 동물은 무엇입니까?


### 부모테이블에서 자식테이블값 조회

In [65]:
q = Question.objects.get(pk=1)
print(type(q), q)
print(q.pk, q.question_text, q.pub_date)

<class 'poll.models.Question'> 좋아하는 색깔은 무엇입니까?
1 좋아하는 색깔은 무엇입니까? 2021-10-05 08:17:35.161348+00:00


In [67]:
# 조회한 Question(부모)의 Choice(자식)들 조회
c_qs = q.choice_set.all()
type(c_qs)

django.db.models.query.QuerySet

In [68]:
for c in c_qs:
    print(c.pk, c.Choice_text, c.vote, sep='\t')

1	검정색	0
2	빨간색	0
8	보라색	2
9	민트색	14


In [69]:
# pk가 1~4인 질문의 질문, 보기의 id, 보기내용 출력
result = Question.objects.filter(pk__range=[1,4])
for q in result:
    print(q.pk, q.question_text)
    choice_qs = q.choice_set.all()
    for choice in choice_qs:
        print("\t", choice.pk, choice.Choice_text, choice.vote)

1 좋아하는 색깔은 무엇입니까?
	 1 검정색 0
	 2 빨간색 0
	 8 보라색 2
	 9 민트색 14
2 좋아하는 동물은 무엇입니까?
	 10 강아지 13
	 11 고양이 6
	 12 원숭이 3
	 13 판다 6
3 가고 싶은 나라를 선택하세요.
	 14 독일 3
	 15 영국 3
	 16 스페인 22
	 17 이탈리아 4
4 싫어하는 동물은 무엇입니까?
	 4 뱀 0
	 5 쥐 3
	 6 고라니 8
	 7 악어 6


# insert/update/delete
- insert/update
    - 모델객체.save()
    - 모델객체가 insert/update할 데이터를 가지고 있다.
- delete
    - 모델객체.delete()
    - 모델객체가 삭제할 데이터의 pk를 가지고 있다.

In [70]:
def get_question_list():
    qs = Question.objects.all()
    for q in qs:
        print(q.pk, q.question_text, q.pub_date, sep='\t')
        
get_question_list()

1	좋아하는 색깔은 무엇입니까?	2021-10-05 08:17:35.161348+00:00
2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00


In [73]:
from datetime import datetime
# insert
# insert할 데이터를 가진 Model 객체를 생성. 객체.save()
new_q = Question(question_text="새로운 질문", pub_date=datetime.now())  # pk(id)는 자동증가이므로 넣지 않음.
print(new_q.pk, new_q.question_text, new_q.pub_date)
new_q.save()

None 새로운 질문 2021-10-06 16:07:20.152474


In [74]:
get_question_list()

1	좋아하는 색깔은 무엇입니까?	2021-10-05 08:17:35.161348+00:00
2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00
6	새로운 질문	2021-10-06 07:07:20.158454+00:00


In [75]:
# update
# update할 데이터를 가진 Model 객체를 생성. 객체.save()
# pk는 테이블에 있는 값을 가지고 있어야 한다.변경되지 않을 값은 원래값을 그대로 가지고 있어야 한다.
update_q = Question(pk=9, question_text="새로운 질문", pub_date=datetime.now())
# id가 9인 질문의 pub_date값을 현재시간으로 update
# update question set pub_date=sysdate where id=9
update_q.save()

C:\Users\Playdata\Anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-10-06 16:13:50.967209) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [76]:
get_question_list()

1	좋아하는 색깔은 무엇입니까?	2021-10-05 08:17:35.161348+00:00
2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00
6	새로운 질문	2021-10-06 07:07:20.158454+00:00
9	새로운 질문	2021-10-06 07:13:51.420704+00:00


In [77]:
q = Question.objects.get(pk=9)
q.question_text = '좋아하는 꽃은 무엇입니까?'
# q.pk, q.question_text, q.pub_date
q.save()

In [78]:
get_question_list()

1	좋아하는 색깔은 무엇입니까?	2021-10-05 08:17:35.161348+00:00
2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00
6	새로운 질문	2021-10-06 07:07:20.158454+00:00
9	좋아하는 꽃은 무엇입니까?	2021-10-06 07:13:51.420704+00:00


In [80]:
# delete
# 삭제할 pk값을 가진 model을 생성. model.delete()
delete_q = Question(pk=9)
print(delete_q.pk, delete_q.question_text, delete_q.pub_date)
delete_q.delete()

9  None


(1, {'poll.Question': 1})

In [81]:
get_question_list()

1	좋아하는 색깔은 무엇입니까?	2021-10-05 08:17:35.161348+00:00
2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00
6	새로운 질문	2021-10-06 07:07:20.158454+00:00


In [83]:
delete_q2 = Question.objects.get(pk=1)
delete_q2.pk, delete_q2.question_text, delete_q2.pub_date

(1,
 '좋아하는 색깔은 무엇입니까?',
 datetime.datetime(2021, 10, 5, 8, 17, 35, 161348, tzinfo=<UTC>))

In [84]:
delete_q2.delete()
# (5: 총 delete된 데이터 수, {'poll_choice':Choice 테이블에서 삭제된 데이터수 "poll.Question":Question 테이블에서 삭제된 데이터 수})

(5, {'poll.Choice': 4, 'poll.Question': 1})

In [85]:
get_question_list()

2	좋아하는 동물은 무엇입니까?	2021-10-05 08:17:48.995562+00:00
3	가고 싶은 나라를 선택하세요.	2021-10-05 08:17:59.136089+00:00
4	싫어하는 동물은 무엇입니까?	2021-10-05 08:18:11.234816+00:00
6	새로운 질문	2021-10-06 07:07:20.158454+00:00


In [86]:
Choice.objects.filter(question=1)

<QuerySet []>

# 직접 SQL문을 작성해서 실행

- 모델.objects.raw("select문")
    - select 문만 가능.

In [88]:
sql = 'SELECT * FROM poll_choice'
rq = Choice.objects.raw(sql)  # select문을 실행하고 그 결과를 Choice객체에 담아준다.
for d in rq:
    print(d.id, d.Choice_text, d.vote)

4 뱀 0
5 쥐 3
6 고라니 8
7 악어 6
10 강아지 13
11 고양이 6
12 원숭이 3
13 판다 6
14 독일 3
15 영국 3
16 스페인 22
17 이탈리아 4
